Import das bibliotecas necessárias


In [1]:
import psycopg2 as pg
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

Configuração da conexão com o banco de dados PostgreSQL

In [2]:
try:
  conn = pg.connect(database="projeto-fbd",host="localhost",port=5432,user='postgres',password='198019')
  print('conexão concluida')
except(Exception) as error:
  #print(error)
  print('conexão não concluida')

conexão concluida


Criação de widgets

In [3]:
try:
    # criando os widgets que serão usados para captarar os dados
    meta_calorica = widgets.Text(disabled=False, placeholder='Meta calórica', value='', description='Meta Calórica')
    quantidade_refeicoes = widgets.Text(disabled=False, description='Quantidade', value='', placeholder='Quantidade de Refeições')
    intervalo_refeicoes = widgets.Text(disabled=False, description='Intervalo', value='',placeholder='Intervalo entre Refeições')
    objetivo = widgets.Text(disabled=False, placeholder='Objetivo da Dieta', value='', description='Objetivo')
    dieta_id = widgets.Text(disabled=False, description='ID', value='', placeholder='ID da Dieta')
    cpf_usuario = widgets.Text(disabled=False, placeholder='CPF: ', value='', description='CPF')
    alimentos = widgets.Textarea(disabled=False, value='', placeholder='Alimentos: ', description='Alimentos')
except Exception as error:
    print('Falha:', error)


Criação dos botões e declaração das suas funções


In [4]:
# Criação dos botões
createButton = widgets.Button(description="Cadastrar")
readButton = widgets.Button(description="Consultar", value='Preencha os campos acima para consultar')
updateButton = widgets.Button(description="Atualizar", value='Preencha os campos acima para atualizar')
deleteButton = widgets.Button(description="Deletar", value='Preencha os campos acima para deletar')
readbuttonPerId = widgets.Button(description="Consultar por ID", value='Preencha o campo ID acima para consultar')
output = widgets.Output()

# Função para criar uma nova tabela Dieta no banco de dados
def onCreateButton(b):
    try:
        # capturando os valores dos widgets
        meta_calorica_value = float(meta_calorica.value)
        quantidade_refeicoes_value = int(quantidade_refeicoes.value)
        intervalo_refeicoes_value = int(intervalo_refeicoes.value)
        objetivo_value = str(objetivo.value)
        dieta_id_value = int(dieta_id.value)
        cpf_usuario_value = str(cpf_usuario.value)
        alimentos_value = str(alimentos.value)

        # criando o cursor para executar as operações no banco de dados
        cursor = conn.cursor()

        # executando o comando para inserir os dados na tabela Dieta
        cursor.execute('INSERT INTO Dieta (Meta_Calorica, Quantidade_de_refeicoes, Intervalo_entre_refeicoes, Objetivo, DietaId, CpfUsuario) VALUES (%s, %s, %s, %s, %s, %s)',
                       (meta_calorica_value, quantidade_refeicoes_value, intervalo_refeicoes_value, objetivo_value, dieta_id_value, cpf_usuario_value))
        
        # executando o comando para inserir os dados na tabela Dieta_Alimentos
        cursor.execute('INSERT INTO Dieta_Alimentos (Alimentos, DietaID) VALUES (%s, %s)', (alimentos_value, dieta_id_value))
        
        # criando um Dataframe para exibir os dados da busca
        df = pd.read_sql_query('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId', conn)

        # exibindo os dados da busca
        try:
            display(df)
        except Exception as error:
            display('Falha ao exibir dados', error)
        
    except (Exception, pg.Error) as error:
        # caso ocorra algum erro, o banco de dados será retornado ao estado anterior
        conn.rollback()
        print('Falha ao inserir dados', error)
    finally:
        # salvando as alterações no banco de dados
        conn.commit()
        cursor.close()

# Função para ler os dados da tabela Dieta no banco de dados
def onReadButton(b):
    try:
        # criando o cursor para executar as operações no banco de dados
        cursor = conn.cursor()

        # executando o comando para ler os dados da tabela Dieta
        cursor.execute('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId')
        
        # criando um Dataframe para exibir os dados da busca
        df = pd.read_sql_query('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId', conn)
        # exibindo os dados da busca
        try:
            display(df)
        except Exception as error:
            display('Falha ao exibir dados', error)

    except (Exception, pg.Error) as error:
        # caso ocorra algum erro, o banco de dados será retornado ao estado anterior
        conn.rollback()
        print('Falha ao ler dados', error)
    
    finally:
        # salvando as alterações no banco de dados
        conn.commit()
        cursor.close()

# Função para atualizar os dados da tabela Dieta no banco de dados
def onUpdateButton(b):
    try:
        # capturando os valores dos widgets
        meta_calorica_value = float(meta_calorica.value)
        quantidade_refeicoes_value = int(quantidade_refeicoes.value)
        intervalo_refeicoes_value = int(intervalo_refeicoes.value)
        objetivo_value = str(objetivo.value)
        dieta_id_value = int(dieta_id.value)
        cpf_usuario_value = str(cpf_usuario.value)
        alimentos_value = str(alimentos.value)

        # criando o cursor para executar as operações no banco de dados
        cursor = conn.cursor()

        # executando o comando para atualizar os dados da tabela Dieta
        cursor.execute('UPDATE Dieta SET Meta_Calorica = %s, Quantidade_de_refeicoes = %s, Intervalo_entre_refeicoes = %s, Objetivo = %s, CpfUsuario = %s WHERE DietaId = %s',
                       (meta_calorica_value, quantidade_refeicoes_value, intervalo_refeicoes_value, objetivo_value, cpf_usuario_value, dieta_id_value))
        
        # executando o comando para atualizar os dados da tabela Dieta_Alimentos
        cursor.execute('UPDATE Dieta_Alimentos SET Alimentos = %s WHERE DietaID = %s', (alimentos_value, dieta_id_value))
        
        # criando um Dataframe para exibir os dados da busca
        df = pd.read_sql_query('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId', conn)
       
       # exibindo os dados da busca
        try:
            display(df)
        except Exception as error:
            display('Falha ao exibir dados', error)
    
    except (Exception, pg.Error) as error:
        # caso ocorra algum erro, o banco de dados será retornado ao estado anterior
        conn.rollback()
        print('Falha ao atualizar dados', error)
    finally:
        # salvando as alterações no banco de dados
        conn.commit()
        cursor.close()

#delete está com bug
def onDeleteButton(b):
    try:
        # capturando os valores dos widgets
        
        dieta_id_value = int(dieta_id.value)

        # criando o cursor para executar as operações no banco de dados
        cursor = conn.cursor()

        # executando o comando para deletar os dados da tabela Dieta e Dieta_Alimentos
        cursor.execute('DELETE FROM Dieta WHERE DietaId = %s', (dieta_id_value,))
        cursor.execute('DELETE FROM Dieta_Alimentos WHERE DietaID = %s', (dieta_id_value,))
        #cursor.execute('', (,))

        # criando um Dataframe para exibir os dados da busca
        df = pd.read_sql_query('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId', conn)

        # exibindo os dados da busca
        try:
            display(df)
        except Exception as error:
            display('Falha ao exibir dados', error)
    except (Exception, pg.Error) as error:
        # caso ocorra algum erro, o banco de dados será retornado ao estado anterior
        conn.rollback()
        print('Falha ao deletar dados', error)
    finally:
        # salvando as alterações no banco de dados
        conn.commit()
        cursor.close()


def onReadButtonPerId(b):
    try:
        # capturando os valores dos widgets
        dieta_id_value = int(dieta_id.value)

        # criando o cursor para executar as operações no banco de dados
        cursor = conn.cursor()
        cursor.execute('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId where d.DietaId = %s', (dieta_id_value,))
        sql_code = 'SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId where d.DietaId = %s' % (dieta_id_value)
        
        # criando um Dataframe para exibir os dados da busca
        df = pd.read_sql_query(sql_code, conn)
        
        # exibindo os dados da busca
        try:
            display(df)
        except Exception as error:
            display('Falha ao exibir dados', error)
    except (Exception, pg.Error) as error:
        # caso ocorra algum erro, o banco de dados será retornado ao estado anterior
        conn.rollback()
        print('Falha ao ler dados', error)
    finally:
        # salvando as alterações no banco de dados
        conn.commit()
        cursor.close()

createButton.on_click(onCreateButton)
readButton.on_click(onReadButton)
updateButton.on_click(onUpdateButton)
deleteButton.on_click(onDeleteButton)
readbuttonPerId.on_click(onReadButtonPerId)

# Display buttons
#display(createButton, readButton, updateButton, deleteButton, output)


Exibir widgets

In [5]:
with output:
    display(meta_calorica,quantidade_refeicoes,intervalo_refeicoes,objetivo,dieta_id,cpf_usuario,alimentos,createButton,readButton,readbuttonPerId,updateButton,deleteButton)
output

Output()